In [1]:
# Important: need to install "backtrader" package/module from PyPI first!

# first, activate cryptoalgowheel conda environment
# then: /anaconda3/envs/cryptoalgowheel/bin/pip install backtrader[plotting]

In [2]:
# *** IMPORTANT: need to install "PyQt5" package from PyPI in order to plot!!!

# first, activate cryptoalgowheel conda environment
# then: /anaconda3/envs/cryptoalgowheel/bin/pip install PyQt5

In [3]:
# ***** NOTICE: the "matplotlib" package in the running environment must be lower than version "3.2.2" (otherwise cannot plot)!!!

The dataset used here is "orcl-2014.txt" from the "datas" folder of the Github repository.

In [4]:
from __future__ import (absolute_import, division, print_function, unicode_literals)

import backtrader as bt
import matplotlib
#import matplotlib.pyplot as plt
import PyQt5

import datetime
import os.path
import sys



In [5]:
print(bt.__version__)
print(matplotlib.__version__)

1.9.76.123
3.2.2


In [6]:
class TestStrategy(bt.Strategy):
    def log(self, txt, dt=None):      #standard log entry print format
        dt = dt or self.datas[0].datetime.date(0)
        print("%s, %s" % (dt.isoformat(), txt))

    def __init__(self):
        #keep a reference to the "close" Line object
        self.dataclose = self.datas[0].close

        self.order = None
    
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return

        if order.status in [order.Completed]:
            if order.isbuy():
                self.log("BUY EXECUTED, %.2f" % order.executed.price)
            elif order.issell():
                self.log("SELL EXECUTED, %.2f" % order.executed.price)
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log("Order Canceled / Margin / Rejected!")

        self.order = None      #signal no pending order now (after execution)

    def next(self):
        self.log("Close, %.2f" % self.dataclose[0])

        if len(self) == 5:  #Suppose we would have a first Buy on the 5th bar
            self.log("BUY CREATE, %.2f" % self.dataclose[0])
            self.order = self.buy()
        if len(self) == 10:  #Suppose we would then have a first Sell on the 10th bar
            self.log("SELL CREATE, %.2f" % self.dataclose[0])
            self.order = self.sell()


if __name__ == '__main__':
    cerebro = bt.Cerebro()

    cerebro.addstrategy(TestStrategy)

    #modpath = os.path.dirname(os.path.abspath(sys.argv[0]))
    #datapath = os.path.join(modpath, '../data/orcl-2014.txt')
    datapath = "../data/orcl-2014.txt"
    #print(modpath)
    print(datapath)

    data = bt.feeds.YahooFinanceCSVData(dataname=datapath, fromdate=datetime.datetime(2014, 1, 2), todate=datetime.datetime(2014, 12, 31), reverse=False)    #*! "reverse" parameter could be set to "False"(default "False") as the pre-downloaded YahooFinance format data has already the right ascending date order

    cerebro.adddata(data)
    cerebro.broker.setcash(100000.0)     #set initial balance in the trading account
    #Suppose: no Commission in this case

    print("Starting Portfolio Value: %.2f" % cerebro.broker.getvalue())

    cerebro.run()

    print("Final Portfolio Value: %.2f" % cerebro.broker.getvalue())
    
    #***!!!NOTICE use "Qt5Agg" matplotlib backend!!!
    matplotlib.use("Qt5Agg")
    #plt.switch_backend("Qt5Agg")
    cerebro.plot(height=30, iplot=False)
    #plt.show()

../data/orcl-2014.txt
Starting Portfolio Value: 100000.00
2014-01-02, Close, 35.17
2014-01-03, Close, 35.07
2014-01-06, Close, 34.93
2014-01-07, Close, 35.29
2014-01-08, Close, 35.17
2014-01-08, BUY CREATE, 35.17
2014-01-09, BUY EXECUTED, 35.29
2014-01-09, Close, 35.10
2014-01-10, Close, 35.53
2014-01-13, Close, 35.19
2014-01-14, Close, 35.62
2014-01-15, Close, 35.81
2014-01-15, SELL CREATE, 35.81
2014-01-16, SELL EXECUTED, 35.79
2014-01-16, Close, 35.70
2014-01-17, Close, 35.62
2014-01-21, Close, 35.53
2014-01-22, Close, 35.41
2014-01-23, Close, 35.57
2014-01-24, Close, 34.60
2014-01-27, Close, 34.02
2014-01-28, Close, 34.59
2014-01-29, Close, 34.47
2014-01-30, Close, 34.87
2014-01-31, Close, 34.40
2014-02-03, Close, 33.41
2014-02-04, Close, 33.53
2014-02-05, Close, 33.52
2014-02-06, Close, 34.23
2014-02-07, Close, 34.67
2014-02-10, Close, 34.77
2014-02-11, Close, 35.28
2014-02-12, Close, 35.49
2014-02-13, Close, 35.82
2014-02-14, Close, 35.41
2014-02-18, Close, 35.40
2014-02-19, Clos

In [7]:
#another Test Strategy 
# (PyAlgoTrade example - using a Simple Moving Average indicator:
# - buy if the close is greater than the moving average
# - sell if the close is smaller than the moving average
#  - (*)only 1 active operation is allowed in the market):

class TestStrategy2(bt.Strategy):

    def log(self, txt, dt=None):
        dt = dt or self.datas[0].datetime.date(0)
        print("%s, %s" % (dt.isoformat(), txt))

    def __init__(self):
        self.dataclose = self.datas[0].close

        self.order = None

        self.sma = bt.indicators.SimpleMovingAverage(self.datas[0], period=5)

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return

        if order.status in [order.Completed]:
            if order.isbuy():
                self.log("BUY EXECUTED, Price: %.2f, Cost: %.2f, Stake: %.2f, Commission: %.2f" % (order.executed.price, order.executed.value, order.executed.size, order.executed.comm))
            else:
                self.log("SELL EXECUTED, Price: %.2f, Cost: %.2f, Stake: %.2f, Commission: %.2f" % (order.executed.price, order.executed.value, order.executed.size, order.executed.comm))

        elif order.status in [ord/er.Canceled, order.Margin, order.Rejected]:
            self.log("Order Canceled / Margin / Rejected")
        
        self.order = None       #remember to signal no pending order now

    def notify_trade(self, trade):
        if not trade.isclosed:
            return
        
        self.log("OPERATION PROFIT, GROSS %.2f, NET %.2f" % (trade.pnl, trade.pnlcomm))

    def next(self):
        self.log("Close, %.2f" % self.dataclose[0])
    
        if self.order:      #if there's a pending order, don't send a 2nd order
            return

        if not self.position:     #if not yet in the market! -- can only Buy first
            if self.dataclose[0] > self.sma[0]:      #if close greater than sma currently --buy!
                self.log("BUY CREATE, %.2f" % self.dataclose[0])
                self.order = self.buy()      #remember to signal there's a BUY pending order!   
        else:    #already in the market
            if self.dataclose[0] < self.sma[0]:      #if close smaller than sma currently -- sell!
                self.log("SELL CREATE, %.2f" % self.dataclose[0])
                self.order = self.sell()     #remember to signal there's a SELL pending order!
            
if __name__ == "__main__":
    cerebro = bt.Cerebro()

    cerebro.addstrategy(TestStrategy2)

    #modpath = os.path.dirname(os.path.abspath(sys.argv[0]))
    #datapath = os.path.join(modpath, "../data/orcl-2014.txt")
    datapath = "../data/orcl-2014.txt"
    #print(modpath)
    print(datapath)

    data = bt.feeds.YahooFinanceCSVData(dataname=datapath, fromdate=datetime.datetime(2014, 1, 2), todate=datetime.datetime(2014, 12, 31), reverse=False)

    cerebro.adddata(data)

    cerebro.broker.setcash(1000.0)

    cerebro.addsizer(bt.sizers.FixedSize, stake=10)      #10 'stake' now instead of default 1 'stake'

    cerebro.broker.setcommission(commission=0.0)      #mark the broker commission rate here

    print("Starting Portfolio Value: %.2f" % cerebro.broker.getvalue())

    cerebro.run()

    print("Final Portfolio Value: %.2f" % cerebro.broker.getvalue())

    #***!!!NOTICE use "Qt5Agg" matplotlib backend!!!
    matplotlib.use("Qt5Agg")
    cerebro.plot(height=30, iplot=False)


../data/orcl-2014.txt
Starting Portfolio Value: 1000.00
2014-01-08, Close, 35.17
2014-01-08, BUY CREATE, 35.17
2014-01-09, BUY EXECUTED, Price: 35.29, Cost: 352.90, Stake: 10.00, Commission: 0.00
2014-01-09, Close, 35.10
2014-01-09, SELL CREATE, 35.10
2014-01-10, SELL EXECUTED, Price: 35.19, Cost: 352.90, Stake: -10.00, Commission: 0.00
2014-01-10, OPERATION PROFIT, GROSS -1.00, NET -1.00
2014-01-10, Close, 35.53
2014-01-10, BUY CREATE, 35.53
2014-01-13, BUY EXECUTED, Price: 35.38, Cost: 353.80, Stake: 10.00, Commission: 0.00
2014-01-13, Close, 35.19
2014-01-13, SELL CREATE, 35.19
2014-01-14, SELL EXECUTED, Price: 35.22, Cost: 353.80, Stake: -10.00, Commission: 0.00
2014-01-14, OPERATION PROFIT, GROSS -1.60, NET -1.60
2014-01-14, Close, 35.62
2014-01-14, BUY CREATE, 35.62
2014-01-15, BUY EXECUTED, Price: 35.61, Cost: 356.10, Stake: 10.00, Commission: 0.00
2014-01-15, Close, 35.81
2014-01-16, Close, 35.70
2014-01-17, Close, 35.62
2014-01-21, Close, 35.53
2014-01-21, SELL CREATE, 35.53
2